# Complete pipeline

1. We create a simple environment in Flatland that has only a single agent.  The agent is given a starting position and a goal.
2. This information is fed to a Clingo encoding, which produces a path for the agent from the start to the goal.
3. We return this path information to Flatland in a manner which can be interpreted and visualized.

## Flatland environment generation

This is Flatland code, not my own.

In [12]:
# import packages
import numpy as np

from flatland.envs.rail_env import RailEnv, RailEnvActions
from flatland.envs.rail_generators import rail_from_manual_specifications_generator
from flatland.envs.rail_generators import rail_from_grid_transition_map
from flatland.envs.rail_generators import sparse_rail_generator
from flatland.envs.observations import TreeObsForRailEnv

from flatland.core.grid.rail_env_grid import RailEnvTransitions
from flatland.core.transition_map import GridTransitionMap

In [13]:
def make_environment():
    """create an environment by specifying the track type of each cell"""
    transitions = RailEnvTransitions()
    cells = transitions.transition_list
    
    # rail map: track types by ID values
    rail_map = np.array(
        [[0] + [16386] + [256]]
        + [[4] + [3089] +[256]]
        , dtype=np.uint16)

    # rail: transitions based on track types
    rail = GridTransitionMap(width=rail_map.shape[1], height=rail_map.shape[0], transitions=transitions)
    
    # optionals
    rail.grid = rail_map
    city_positions = [(0,1), (3,0)]
    train_stations = [[((0,1),0)], [((3,0),0)],]
    city_orientations = [0,0]
    agents_hints = {
        'city_positions': city_positions,
        'train_stations': train_stations,
        'city_orientations': city_orientations
    }
    optionals = {'agents_hints': agents_hints}
    return rail, rail_map, optionals

In [14]:
rail, rail_map, optionals = make_environment()
rail_gen = rail_from_grid_transition_map(rail_map=rail, optionals=optionals)
hand_crafted_env = RailEnv(width=3, height=2, rail_generator=rail_gen, number_of_agents=1, obs_builder_object=TreeObsForRailEnv(max_depth=0))


### Example output of Flatland's `rail_map`

In [15]:
rail_map


array([[    0, 16386,   256],
       [    4,  3089,   256]], dtype=uint16)

## Clingo pathfinding encoding

In [16]:
from pathlib import Path

### `clingo_map` class

In [17]:
class clingo_map():
    """an alternative representation of a Flatland map that clingo can process"""
    
    def __init__(self, rail_map):
        self.clingo_str = ""
        self.mapping = {}
        
        row_num = len(rail_map) - 1
        for row in rail_map:
            for col,cval in enumerate(row):
                self.clingo_str += "cell(({},{}), {}). ".format(col,row_num,cval)
                self.mapping[(col,row_num)] = cval
            row_num -= 1


In [18]:
env = clingo_map(rail_map)
print(env.clingo_str) # take a look


cell((0,1), 0). cell((1,1), 16386). cell((2,1), 256). cell((0,0), 4). cell((1,0), 3089). cell((2,0), 256). 


In [19]:
import clingo

def on_model(m):
    """print results upon solving"""
    print(m)

In [20]:
# clingo API
ctl = clingo.Control()

# load the map into clingo
ctl.add("base", [], env.clingo_str)

# load the types.lp reference file
types = Path('../attempt_05_full/types.lp').read_text()
ctl.add("base", [], types)

# load the encoding
encoding = Path('../attempt_05_full/encoding.lp').read_text() 
ctl.add("base", [], encoding)

# ground & solve
ctl.ground([("base", [])])
ctl.solve(on_model=on_model)

offset((n,f),(0,-1),s) offset((w,r),(0,-1),s) offset((e,l),(0,-1),s) offset((e,f),(-1,0),w) offset((n,r),(-1,0),w) offset((s,l),(-1,0),w) offset((s,f),(0,1),n) offset((w,l),(0,1),n) offset((e,r),(0,1),n) offset((w,f),(1,0),e) offset((n,l),(1,0),e) offset((s,r),(1,0),e) swap(n,s) swap(s,n) swap(e,w) swap(w,e) move(f) move(l) move(r) move(wait) type(32800,(n,f)) type(32800,(s,f)) type(1025,(w,f)) type(1025,(e,f)) type(4608,(s,l)) type(4608,(w,r)) type(16386,(e,l)) type(16386,(s,r)) type(72,(n,l)) type(72,(e,r)) type(2064,(w,l)) type(2064,(n,r)) type(37408,(s,l)) type(37408,(w,r)) type(37408,(s,f)) type(37408,(n,f)) type(17411,(e,l)) type(17411,(s,r)) type(17411,(e,f)) type(17411,(w,f)) type(32872,(n,l)) type(32872,(e,r)) type(32872,(n,f)) type(32872,(s,f)) type(3089,(w,l)) type(3089,(n,r)) type(3089,(w,f)) type(3089,(e,f)) type(49186,(s,r)) type(49186,(e,l)) type(49186,(s,f)) type(49186,(n,f)) type(1097,(e,r)) type(1097,(n,l)) type(1097,(e,f)) type(1097,(w,f)) type(34864,(n,r)) type(3486

SolveResult(1)

**Expected result**

`at(cell(0,0),0,e) at(cell(1,0),1,e) at(cell(1,1),2,n) at(cell(2,1),3,e)`

In [141]:
env.mapping

{(0, 1): 0, (1, 1): 16386, (2, 1): 256, (0, 0): 4, (1, 0): 3089, (2, 0): 256}

### Translate solution → actions

In [150]:
# translate clingo solution into moves
switch_ids = [37408,17411,32872,3089,49186,1097,34864,5633,38433,50211,33897,35889,38505,52275,20994,16458,2136,6672]

def is_switch(loc, env): #env.mapping
    """
    given a location and its environment, determine whether the cell has a switch
    """
    return env[loc] in switch_ids

def next_move(dirs, locs, env):
    """
    given:
        * a current cardinal direction and the following one, 
        * a current location and the following one
        * an environment
    return what move the train should make
    """
    if locs[0] == locs[1]:
        return "STOP_MOVING"
    elif dirs[0] == dirs[1]:
        return "MOVE_FORWARD"
    #elif 


In [148]:
is_switch((1,0),env.mapping)

True

## Flatland visualization

In [3]:
# code here

In [10]:
%%clingo 0

p(1). p(2). p(3).
{ holds(q(X)) : p(X) }.
holds(a) :- holds(q(1)).
#show.
#show X : holds(X).

clingo version 5.6.2
Reading from stdin
Solving...
Answer: 1

Answer: 2
q(2)
Answer: 3
q(3)
Answer: 4
q(2) q(3)
Answer: 5
q(1) a
Answer: 6
q(1) q(3) a
Answer: 7
q(1) q(2) a
Answer: 8
q(1) q(2) q(3) a
SATISFIABLE

Models       : 8
Calls        : 1
Time         : 0.008s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.002s


In [103]:
%%clingo 0

% given
at(cell(0,0),0,e).
at(cell(1,0),1,e).
at(cell(1,1),2,n).
at(cell(2,1),3,e).

% MOVEMENT %
% determine whether the train is in motion or remaining in its current location %

stationary(T0,T1) :- at(cell(X,Y),T0,_), at(cell(X,Y),T1,_), T1-T0=1, T0<T1.
moving(T0,T1) :- at(cell(X0,Y0),T0,_), at(cell(X1,Y1),T1,_), T1-T0=1, T0<T1, (X0+Y0)<(X1+Y1).
    
% ROTATION %
% what is the rotation?
%orientation
orient(T,0) :- at(_,T,n).
orient(T,1) :- at(_,T,e).
orient(T,2) :- at(_,T,s).
orient(T,3) :- at(_,T,w).
    
%rotate_0(T0,T1) :- at(_,T0,D), at(_,T1,D), T1-T0=1, T0<T1.
    
%rotate_0(T0,T1) :- at(_,T0,_), at(_,T1,_), orient(T0,D0), orient(T1,D1), T1-T0=1, T0<T1, D1=D0.
%rotate_90(T0,T1) :- at(_,T0,_), at(_,T1,_), orient(T0,D0), orient(T1,D1), T1-T0=1, T0<T1, 4\(D1-D0)=1.
%rotate_180(T0,T1) :- at(_,T0,_), at(_,T1,_), orient(T0,D0), orient(T1,D1), T1-T0=1, T0<T1, 4\(D1-D0)=2.
%rotate_270(T0,T1) :- at(_,T0,_), at(_,T1,_), orient(T0,D0), orient(T1,D1), T1-T0=1, T0<T1, 4\(D1-D0)=3.
    
%rotate(T0,T1,R) :- at(_,T0,_), at(_,T1,_), orient(T0,D0), orient(T1,D1), T1-T0=1, T0<T1, Diff = D1-D0, R=4 \ Diff.

%#show.
#show stationary/2.
#show moving/2.
%#show rotate/3.
%#show orientation/2.
%#show stop_moving(T) : a, at(cell(X,Y),T,D).
%#show move_left(T) : b, at(cell(X,Y),T,D).
%#show move_right(T) : c, at(cell(X,Y),T,D).
%#show move_forward(T) : not a, not b, not c, at(cell(X,Y),T,D).

clingo version 5.6.2
Reading from stdin
Solving...
Answer: 1
moving(0,1) moving(1,2) moving(2,3)
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.001s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.001s


In [136]:
%%clingo 0

mod(X,Y,Z) :- X=12, Y=4, Z=X\Y.

clingo version 5.6.2
Reading from stdin
Solving...
Answer: 1
mod(12,4,0)
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.001s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.000s


In [143]:
%%clingo 0

% given
at(cell(0,0),0,e).
at(cell(1,0),1,e).
%at(cell(1,1),2,n).
at(cell(1,1),2,n).
at(cell(1,1),3,n).
at(cell(2,1),4,e).



% MOVEMENTb
movement(T0,M) :- at(cell(X0,Y0), T0, _), at(cell(X1,Y1), T1, _), T1>T0, T1-T0=1, M=|(X1+Y1)-(X0+Y0)|.

% ROTATION
map(0,n). map(1,e). map(2,s). map(3,w).
or(T,Mapping) :- at(_,T,Direction), map(Mapping,Direction).
rotation(T0,R) :- or(T0,M0), or(T1,M1), T1>T0, T1-T0=1, R=M1-M0\4.
    
% TRACK TYPE


#show movement/2.
#show rotation/2.
#show stop_moving(T) : rotation(T,R), R = 0, movement(T,M), M = 0.

clingo version 5.6.2
Reading from stdin
Solving...
Answer: 1
movement(0,1) movement(1,1) movement(2,0) movement(3,1) rotation(0,0) rotation(1,-1) rotation(2,0) rotation(3,1) stop_moving(2)
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.012s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.003s


In [183]:
%%clingo 0 test_env.lp -

% path             Expected results:
at(cell(0,0),0,e). %MOVE_FORWARD
at(cell(1,0),1,e). %MOVE_LEFT
at(cell(1,1),2,n). %STOP_MOVING
at(cell(1,1),3,n). %MOVE_FORWARD
at(cell(2,1),4,e).

% ROTATION
map(0,n). map(1,e). map(2,s). map(3,w).
orient(T,Map) :- at(_,T,Dir), map(Map,Dir).
rotation(T0,R) :- orient(T0,M0), orient(T1,M1), T1>T0, T1-T0=1, R=M1-M0\4.
    
% TRACK TYPE
type(T,switch) :- at(cell(X,Y),T,_), cell((X,Y),ID), ID = (37408;17411;32872;3089;49186;1097;34864;5633;38433;50211;33897;35889;38505;52275;20994;16458;2136;6672).
type(T,deadend) :- at(cell(X,Y),T,_), cell((X,Y),ID), ID = (8192;4;128;256).
type(T,nonswitch) :- at(cell(X,Y),T,_), not type(T,switch), not type(T,deadend).
    
% MOVEMENT
movement(T0,M) :- at(cell(X0,Y0), T0, _), at(cell(X1,Y1), T1, _), T1>T0, T1-T0=1, M=|(X1+Y1)-(X0+Y0)|.
   

#show.
#show stop_moving(T0) : at(C,T0,D), at(C,T1,D), T1-T0=1.
#show move_left(T) : rotation(T,-1), type(T,switch).
#show move_right(T): rotation(T,1), type(T,switch).
#show move_forward(T): movement(T,1), not type(T,switch).
%#show move_forward(T): movement(T,1), rotation(T,0).
%#show move_forward(T): rotation(T,R), |R|=1, type(T,nonswitch).
#show move_forward(T): rotation(T,R), |R|=2, type(T,deadend).

clingo version 5.6.2
Reading from test_env.lp ...
Solving...
Answer: 1
move_forward(0) move_forward(3) move_left(1) stop_moving(2)
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.012s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.004s


In [110]:
%%clingo 0

letter(a;b;c).

clingo version 5.6.2
Reading from stdin
Solving...
Answer: 1
letter(a) letter(b) letter(c)
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.008s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.001s


In [145]:
%%clingo 0 types.lp

a.

clingo version 5.6.2
Reading from types.lp
Solving...
Answer: 1
move(f) move(l) move(r) move(wait) dir(e) dir(w) dir(n) dir(s) swap(n,s) swap(s,n) swap(e,w) swap(w,e) type(32800,(n,f)) type(32800,(s,f)) type(1025,(w,f)) type(1025,(e,f)) type(4608,(s,l)) type(4608,(w,r)) type(16386,(e,l)) type(16386,(s,r)) type(72,(n,l)) type(72,(e,r)) type(2064,(w,l)) type(2064,(n,r)) type(37408,(s,l)) type(37408,(w,r)) type(37408,(s,f)) type(37408,(n,f)) type(17411,(e,l)) type(17411,(s,r)) type(17411,(e,f)) type(17411,(w,f)) type(32872,(n,l)) type(32872,(e,r)) type(32872,(n,f)) type(32872,(s,f)) type(3089,(w,l)) type(3089,(n,r)) type(3089,(w,f)) type(3089,(e,f)) type(49186,(s,r)) type(49186,(e,l)) type(49186,(s,f)) type(49186,(n,f)) type(1097,(e,r)) type(1097,(n,l)) type(1097,(e,f)) type(1097,(w,f)) type(34864,(n,r)) type(34864,(w,l)) type(34864,(n,f)) type(34864,(s,f)) type(5633,(w,r)) type(5633,(s,l)) type(5633,(w,f)) type(5633,(e,f)) type(33825,(n,f)) type(33825,(s,f)) type(33825,(e,f)) type(33825,